In [1]:
import numpy as np
import cv2
import os

In [2]:
glass = cv2.imread('Train/glasses.png',-1)
mustache = cv2.imread('Train/mustache.png',-1)

In [3]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_mcs_eyepair_big.xml')
nose_cascade = cv2.CascadeClassifier('haarcascade_mcs_nose.xml')

In [4]:
cap = cv2.VideoCapture(0) 
while(True):
    ret,gray_frame = cap.read()
    if ret==False:
        continue
        
    img = gray_frame
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces=face_cascade.detectMultiScale(gray,1.3,5)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
    for (x,y,w,h) in faces:
        rg = gray[y:y+h, x:x+w]
        face = img[y:y+h,x:x+w]
        eyes = eye_cascade.detectMultiScale(rg, 1.3, 5)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
        for (ex,ey,ew,eh) in eyes:
            glass = cv2.resize(glass.copy(), (ew,eh))
            glasses2 = cv2.resize(glass.copy(),(int(1.1*ew),int(2.5*eh)))
            eye = rg[ey:ey+eh, ex:ex+ew]
            gw,gh,gc = glasses2.shape
            for i in range(0,gw):
                for j in range(0,gh):
                    if glasses2[i,j][3]!=0:
                        face[ ey - int(eh/1.5)+i, int(ex)+j] = glasses2[i,j]
                        
        noses = nose_cascade.detectMultiScale(rg, 1.3, 5)
        for (nx,ny,nw,nh) in noses:
            mustache = cv2.resize(mustache.copy(),(nw,nh))
            mustache2 = cv2.resize(mustache.copy(),(nw,int(0.5*ny)))
            nose = rg[ny:ny+nh, nx:nx+nw]
            mw,mh,mc = mustache2.shape
            for i in range(0,mw):
                for j in range(0,mh):
                    if mustache2[i,j][3]!=0:
                        face[ny + int(nh/1.5) + i, nx+j] = mustache2[i,j]
    cv2.imshow("frame",img)
    key=cv2.waitKey(1) & 0xFF

    if key==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()